# FEATURE ENGINEERING

This notebook logically follows the preprocessing step and performs feature engineering for modeling or further analysis.

✔ What it does:

- Loads the cleaned data from the previous step (pre-processed.csv).

Creates new features:
- Price Change %
- Target (binary label: will price rise next day?)
- Date features: weekday, month, quarter
- Rolling averages (MA_5, MA_10)
- Daily Return and log of Adjusted Close price


In [1]:
import os
import pandas as pd
import numpy as np

file_path = "data\PREPROCESSING\merged_stock_income.csv" 

df_merged = pd.read_csv(file_path)

df_merged.info()

<>:5: SyntaxWarning: invalid escape sequence '\P'
<>:5: SyntaxWarning: invalid escape sequence '\P'
C:\Users\SABIO\AppData\Local\Temp\ipykernel_13272\3320304327.py:5: SyntaxWarning: invalid escape sequence '\P'
  file_path = "data\PREPROCESSING\merged_stock_income.csv"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4499390 entries, 0 to 4499389
Data columns (total 32 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               float64
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  Re

In [2]:
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common)
0,GOOG,2019-04-26,18,63.45,63.65,63.02,63.61,63.31,24828560,1.388488e+10,...,3.592800e+10,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10
1,SPWH,2019-04-26,1249258,4.67,4.71,4.59,4.66,4.66,169451,4.297878e+07,...,3.346400e+07,-7.995000e+06,-7995000.0,2.546900e+07,2.218000e+06,2.546900e+07,-5.254000e+06,2.021500e+07,2.021500e+07,2.021500e+07
2,TLRA,2019-04-26,646927,7.23,7.42,7.07,7.34,7.34,587759,4.559242e+07,...,-1.291600e+07,4.385000e+06,0.0,-8.531000e+06,0.000000e+00,-8.531000e+06,-4.760000e+05,-9.007000e+06,-9.007000e+06,-9.007000e+06
3,EAST,2019-04-26,646929,119.00,121.00,119.00,120.00,120.00,723,4.555320e+05,...,-9.968000e+06,-3.178000e+06,-508000.0,-1.314600e+07,1.500000e+04,-1.313100e+07,-5.606000e+06,-1.313100e+07,-1.690800e+07,-1.690800e+07
4,RGR,2019-04-26,1248958,56.56,57.86,56.56,57.39,42.11,155235,1.745802e+07,...,3.937500e+07,3.652000e+06,2402000.0,4.302700e+07,-4.181265e+08,4.302700e+07,-1.073600e+07,3.229100e+07,3.229100e+07,3.229100e+07


New Columns: 

✅ Price Change % → Added daily price change % calculation (1 day lag)

✅ Target → Created Target column for market movement prediction

✅ Weekday (Weekday) → Categorical feature (Monday-Sunday) for classification models.

✅ Month (Month) → Categorical month number (1-12) for classification models.

✅ Quarter (Quarter) → Categorical feature (1-4) for classification models.

✅ Daily Returns (Daily_Return) → Percentage change in adjusted close price, useful for financial modeling.

✅ Moving Averages (MA_5, MA_10) → 5-day and 10-day moving averages for trend detection.

✅ Log Returns (Log_Returns) → Helps in normalizing price data for time series models.


In [3]:
df_merged["Price Change %"] = df_merged["Close"].pct_change() * 100  # Use correct column name

df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %
0,GOOG,2019-04-26,18,63.45,63.65,63.02,63.61,63.31,24828560,1.388488e+10,...,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,NaN
1,SPWH,2019-04-26,1249258,4.67,4.71,4.59,4.66,4.66,169451,4.297878e+07,...,-7.995000e+06,-7995000.0,2.546900e+07,2.218000e+06,2.546900e+07,-5.254000e+06,2.021500e+07,2.021500e+07,2.021500e+07,-92.674108
2,TLRA,2019-04-26,646927,7.23,7.42,7.07,7.34,7.34,587759,4.559242e+07,...,4.385000e+06,0.0,-8.531000e+06,0.000000e+00,-8.531000e+06,-4.760000e+05,-9.007000e+06,-9.007000e+06,-9.007000e+06,57.510730
3,EAST,2019-04-26,646929,119.00,121.00,119.00,120.00,120.00,723,4.555320e+05,...,-3.178000e+06,-508000.0,-1.314600e+07,1.500000e+04,-1.313100e+07,-5.606000e+06,-1.313100e+07,-1.690800e+07,-1.690800e+07,1534.877384
4,RGR,2019-04-26,1248958,56.56,57.86,56.56,57.39,42.11,155235,1.745802e+07,...,3.652000e+06,2402000.0,4.302700e+07,-4.181265e+08,4.302700e+07,-1.073600e+07,3.229100e+07,3.229100e+07,3.229100e+07,-52.175000


Create the Target Variable for ML
To predict whether the price will go up or down, shift the adjusted close price:

In [4]:
df_merged["Target"] = (df_merged["Close"].shift(-1) > df_merged["Close"]).astype(int)  # 1 if price rises, 0 if falls
df_merged.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,GOOG,2019-04-26,18,63.45,63.65,63.02,63.61,63.31,24828560,1.388488e+10,...,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,NaN,0
1,SPWH,2019-04-26,1249258,4.67,4.71,4.59,4.66,4.66,169451,4.297878e+07,...,-7995000.0,2.546900e+07,2.218000e+06,2.546900e+07,-5.254000e+06,2.021500e+07,2.021500e+07,2.021500e+07,-92.674108,1
2,TLRA,2019-04-26,646927,7.23,7.42,7.07,7.34,7.34,587759,4.559242e+07,...,0.0,-8.531000e+06,0.000000e+00,-8.531000e+06,-4.760000e+05,-9.007000e+06,-9.007000e+06,-9.007000e+06,57.510730,1
3,EAST,2019-04-26,646929,119.00,121.00,119.00,120.00,120.00,723,4.555320e+05,...,-508000.0,-1.314600e+07,1.500000e+04,-1.313100e+07,-5.606000e+06,-1.313100e+07,-1.690800e+07,-1.690800e+07,1534.877384,0
4,RGR,2019-04-26,1248958,56.56,57.86,56.56,57.39,42.11,155235,1.745802e+07,...,2402000.0,4.302700e+07,-4.181265e+08,4.302700e+07,-1.073600e+07,3.229100e+07,3.229100e+07,3.229100e+07,-52.175000,0


Handling Null Values: Since .pct_change() creates NaN values for the first row, fill them:

In [5]:
df_merged["Price Change %"].fillna(0, inplace=True)
df_merged["Target"].fillna(0, inplace=True)
df_merged.head()

C:\Users\SABIO\AppData\Local\Temp\ipykernel_13272\287450061.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["Price Change %"].fillna(0, inplace=True)
C:\Users\SABIO\AppData\Local\Temp\ipykernel_13272\287450061.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,...,"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target
0,GOOG,2019-04-26,18,63.45,63.65,63.02,63.61,63.31,24828560,1.388488e+10,...,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,0.000000,0
1,SPWH,2019-04-26,1249258,4.67,4.71,4.59,4.66,4.66,169451,4.297878e+07,...,-7995000.0,2.546900e+07,2.218000e+06,2.546900e+07,-5.254000e+06,2.021500e+07,2.021500e+07,2.021500e+07,-92.674108,1
2,TLRA,2019-04-26,646927,7.23,7.42,7.07,7.34,7.34,587759,4.559242e+07,...,0.0,-8.531000e+06,0.000000e+00,-8.531000e+06,-4.760000e+05,-9.007000e+06,-9.007000e+06,-9.007000e+06,57.510730,1
3,EAST,2019-04-26,646929,119.00,121.00,119.00,120.00,120.00,723,4.555320e+05,...,-508000.0,-1.314600e+07,1.500000e+04,-1.313100e+07,-5.606000e+06,-1.313100e+07,-1.690800e+07,-1.690800e+07,1534.877384,0
4,RGR,2019-04-26,1248958,56.56,57.86,56.56,57.39,42.11,155235,1.745802e+07,...,2402000.0,4.302700e+07,-4.181265e+08,4.302700e+07,-1.073600e+07,3.229100e+07,3.229100e+07,3.229100e+07,-52.175000,0


In [6]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Show DataFrame info
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4499390 entries, 0 to 4499389
Data columns (total 34 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Ticker                                    object 
 1   Date                                      object 
 2   SimFinId                                  int64  
 3   Open                                      float64
 4   High                                      float64
 5   Low                                       float64
 6   Close                                     float64
 7   Adj. Close                                float64
 8   Volume                                    int64  
 9   Shares Outstanding                        float64
 10  Currency                                  object 
 11  Fiscal Year                               float64
 12  Fiscal Period                             object 
 13  Publish Date                              object 
 14  Re

In [7]:
df_merged.columns

Index(['Ticker', 'Date', 'SimFinId', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Shares Outstanding', 'Currency', 'Fiscal Year',
       'Fiscal Period', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expenses', 'Selling, General & Administrative',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations', 'Net Income',
       'Net Income (Common)', 'Price Change %', 'Target'],
      dtype='object')

Weekday column

In [8]:
df_merged['Date'] = pd.to_datetime(df_merged['Date']) # converting the "Date" column to a Datetime Object (double checking)

In [9]:
# 1. Weekday (Categorical for Classification)
df_merged['Weekday'] = df_merged['Date'].dt.day_name()  # 'Monday', 'Tuesday', etc.

Month Column

In [10]:
# 2. Month (Categorical for Classification)
df_merged['Month'] = df_merged['Date'].dt.month  # 1 (Jan) to 12 (Dec)

Quarter Column

In [11]:
# 3. Quarter (Categorical for Classification)
df_merged['Quarter'] = df_merged['Date'].dt.quarter  # 1 to 4

Daily Returns (Percentage Change)

In [12]:
df_merged['Daily_Return'] = df_merged['Close'].pct_change() * 100  # Convert to %

Moving Averages (5-day and 10-day)

In [13]:
df_merged['MA_5'] = df_merged['Close'].rolling(window=5, min_periods=1).mean()
df_merged['MA_10'] = df_merged['Close'].rolling(window=10, min_periods=1).mean()

Log Returns for Time Series Modeling

In [14]:
# Compute daily log-returns
df_merged['Log_Returns'] = np.log(df_merged['Close'] / df_merged['Close'].shift(1))

c:\Users\SABIO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
df_merged.set_index('Date', inplace=True)

In [16]:
df_merged.head()

,Ticker,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Shares Outstanding,Currency,Fiscal Year,Fiscal Period,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income,Net Income (Common),Price Change %,Target,Weekday,Month,Quarter,Daily_Return,MA_5,MA_10,Log_Returns
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-26,GOOG,18,63.45,63.65,63.02,63.61,63.31,24828560,1.388488e+10,USD,2019.0,FY,2020-02-04,2022-02-02,1.385192e+10,1.490166e+10,1.618570e+11,-7.189600e+10,8.996100e+10,-5.403300e+10,-2.801500e+10,3.592800e+10,5.394000e+09,0.0,4.132200e+10,-1.697000e+09,3.962500e+10,-5.282000e+09,3.434300e+10,3.434300e+10,3.434300e+10,0.000000,0,Friday,4,2,NaN,63.610000,63.610000,NaN
2019-04-26,SPWH,1249258,4.67,4.71,4.59,4.66,4.66,169451,4.297878e+07,USD,2019.0,FY,2020-04-09,2022-03-30,4.316600e+07,4.358800e+07,8.864010e+08,-5.897680e+08,2.966330e+08,-2.631690e+08,-2.631690e+08,3.346400e+07,-7.995000e+06,-7995000.0,2.546900e+07,2.218000e+06,2.546900e+07,-5.254000e+06,2.021500e+07,2.021500e+07,2.021500e+07,-92.674108,1,Friday,4,2,-92.674108,34.135000,34.135000,-2.613755
2019-04-26,TLRA,646927,7.23,7.42,7.07,7.34,7.34,587759,4.559242e+07,USD,2019.0,FY,2020-03-16,2020-03-16,4.586450e+07,4.586450e+07,6.803800e+07,-1.362500e+07,5.441300e+07,-6.732900e+07,-5.470300e+07,-1.291600e+07,4.385000e+06,0.0,-8.531000e+06,0.000000e+00,-8.531000e+06,-4.760000e+05,-9.007000e+06,-9.007000e+06,-9.007000e+06,57.510730,1,Friday,4,2,57.510730,25.203333,25.203333,0.454323
2019-04-26,EAST,646929,119.00,121.00,119.00,120.00,120.00,723,4.555320e+05,USD,2019.0,FY,2020-03-30,2021-03-31,4.637850e+05,4.637850e+05,1.162600e+07,-7.567000e+06,4.059000e+06,-1.402700e+07,-1.402700e+07,-9.968000e+06,-3.178000e+06,-508000.0,-1.314600e+07,1.500000e+04,-1.313100e+07,-5.606000e+06,-1.313100e+07,-1.690800e+07,-1.690800e+07,1534.877384,0,Friday,4,2,1534.877384,48.902500,48.902500,2.794153
2019-04-26,RGR,1248958,56.56,57.86,56.56,57.39,42.11,155235,1.745802e+07,USD,2019.0,FY,2020-02-20,2022-02-23,1.746142e+07,1.777883e+07,4.105060e+08,-3.109580e+08,9.954800e+07,-6.017300e+07,-6.011900e+07,3.937500e+07,3.652000e+06,2402000.0,4.302700e+07,-4.181265e+08,4.302700e+07,-1.073600e+07,3.229100e+07,3.229100e+07,3.229100e+07,-52.175000,0,Friday,4,2,-52.175000,50.600000,50.600000,-0.737622


In [17]:
df_merged['Log_Returns'].dropna(inplace=True)

In [18]:
df_merged['Log_Returns'].describe()

c:\Users\SABIO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


count    4.499370e+06
mean              NaN
std               NaN
min              -inf
25%     -1.418076e+00
50%      0.000000e+00
75%      1.417486e+00
max               inf
Name: Log_Returns, dtype: float64

In [19]:
# Replace inf/-inf with NaN
df_merged['Log_Returns'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop missing values
df_merged.dropna(subset=['Log_Returns'], inplace=True)

C:\Users\SABIO\AppData\Local\Temp\ipykernel_13272\3151199279.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['Log_Returns'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [20]:
df_merged['Log_Returns'].describe()

count    4.480692e+06
mean     1.071102e-05
std      2.295998e+00
min     -1.781986e+01
25%     -1.410989e+00
50%      0.000000e+00
75%      1.410509e+00
max      1.775023e+01
Name: Log_Returns, dtype: float64

In [21]:
df_merged['Close'].isna().sum()
df_merged['Close'].eq(0).sum()

np.int64(0)

In [22]:
# Save enriched dataset
os.makedirs("data/ENRICH", exist_ok=True)
df_merged.to_csv("data/ENRICH/merged_stock_income.csv", index=True)